# Data Engineer Challenge 

In [ ]:
from pathlib import Path

from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time


In [ ]:
%load_ext memory_profiler

In [ ]:
file_path = (
	Path().cwd().parent.absolute() / "data" / "farmers-protest-tweets-2021-2-4.json"
)

## Problema 1

### Comparación tiempo de ejecución

In [ ]:
%%timeit
q1_time(file_path)


In [ ]:
%%timeit
q1_memory(file_path)

Breve análisis...

### Comparación de memoria

In [ ]:
%%memit
q1_time(file_path)

In [ ]:
%%memit
q1_memory(file_path)

## Problema 2

### Comparación tiempo ejecución

In [ ]:
%%timeit
q2_time(file_path)

In [ ]:
%%timeit
q2_memory(file_path)

Breve análisis

### Comparación uso de memoria

In [ ]:
%%memit
q2_time(file_path)

In [ ]:
%%memit
q2_memory(file_path)

## Problema 3

### Comparación tiempo ejecución

In [ ]:
%%timeit
q3_time(file_path)

In [ ]:
%%timeit
q3_memory(file_path)

Breve análisis problema 3 tiempo

### Comparación uso de memoria

In [ ]:
%%memit
q3_time(file_path)

In [ ]:
%%memit
q3_memory(file_path)

Breve análisis